# Data_Cleaninig__
___
___



The web-scrape of this data was partially monitored and throughout that process I took note of persistently irrelevant results concerning Walmart and Pets. 
The initial exploration was conducted by examining proportions of posts containing lists of keywords to check for topic overlap and identify ways to reduce our dataframes dimensionality by dropping irrelevant content.  

|---|Homeless|Wal-mart|Pet's|
|---|---|---|---|
|posts|305|57|128|
|P_t|28%|5%|12%|
|Containing Homeless|---| 1 | 27 |


|Homeless| Wal-Mart  | Pet's |
|---|---|---|   
|'homeless'|'walmart'|'cat'|
|'unhoused'|'wal-mart'|'dog'|
|'vagrant'||'pet'|
|'squatter'|
|'sos'|
|'encampment'|
 

## Wal-mart
Nextdoor.com has a marketing partnership with Walmart that is likely to blame for the extraneous content of this nature. The results of our cleaning and exploration below supported the decision to remove all posts containing mention of Walmart from the data before applying the topic modeling. This lends a semi-supervised quality to our process, but removing advertisements or brand mentions in this way would not be difficult at scale. 

## Pet's
Post's concerning pets were very consistent, we will need to be careful with dealing with this topic as pet ownership is a frequently discussed topic in discussions of homelessness in the greater Denver area. Our exploration below confirmed our suspicions of this. With 27% of posts containing mention of pets also including referenced to homelessness. We will need to do a round of topic modeling in order to filter this and any remaining irrelevant content.

___
___

In [1]:
from myfunct import *
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [2]:
user_1_posts = pd.read_csv('../data/user_data/vol_1_posts.csv', index_col = None)
user_1_com = pd.read_csv('../data/user_data/vol_1_comments.csv')

In [3]:
user_2_posts = pd.read_csv('../data/user_data/vol_2_posts.csv', index_col = None)
user_2_com = pd.read_csv('../data/user_data/vol_1_comments.csv')

___
___
# User Posts
___
___

## User 1 

In [4]:
print(f'{user_1_posts.shape[0]} posts from User 1.')
user_1_posts.head(3)

455 posts from User 1.


,Unnamed: 0,post_id,author,location,date,post
0,0,186113992,News,NaN,NaN,Park Hill residents sue to stop legal homeless...
1,1,185885904,News,NaN,NaN,Regis University will host Denver's next safe ...
2,2,183301151,Sara Lange,South Park Hill,12 Apr,Homeless Encampment Near You! If you thought t...


## User 2 

In [5]:
print(f'{user_2_posts.shape[0]} posts from User 2.')
user_2_posts.head(3)

629 posts from User 2.


,Unnamed: 0,post_id,author,location,date,post
0,0,186113992,News,NaN,NaN,Park Hill residents sue to stop legal homeless...
1,1,185885904,News,NaN,NaN,Regis University will host Denver's next safe ...
2,2,183301151,Sara Lange,South Park Hill,12 Apr,Homeless Encampment Near You! If you thought t...


___
___
## Merging User Posts
___
___

Notice that the post Id's change format from one dataframe to the next, this is a byproduct of Nextdoor.com loading differently for different users. The two users we pulled from have 1 neighbourhood in their Nextdoor Regions that overlaps, we can see from the first post in either dataset that we do in fact have some duplicate to take care of. We wont be able to use the post id column for this, but that is ok, we have clean texts so any duplicate posts should match exactly.

In [6]:
print(f'Shape Before Duplicate Dropping: {user_1_posts.shape[0] + user_2_posts.shape[0]}')

Shape Before Duplicate Dropping: 1084


In [7]:
user_posts = [user_1_posts, user_2_posts]
posts_df = pd.concat(user_posts, ignore_index=True)
posts_df.drop_duplicates(subset = 'post_id', keep='last', inplace = True)

In [8]:
print(f'Shape After Duplicate Dropping: {posts_df.shape[0]}')

Shape After Duplicate Dropping: 933


In [9]:
nlp = spacy.load('en_core_web_lg', diasble = ['parser','ner'])

In [10]:
posts = posts_df['post']

In [11]:
lemma_posts = lemmatization(posts)

In [12]:
posts_df['lemma'] = [ post for post in lemma_posts]

In [13]:
posts_df

,Unnamed: 0,post_id,author,location,date,post,lemma
4,4,182941686,Beth Daoud,Congress Park,9 Apr,Squatters Camp. Am I the only one outraged by ...,outrage new squatter camp go 14th let allow ha...
5,5,184828873,Darja Pisorn,Mayfair,26 Apr,Strange squirrel activity. Hey neighbors! Hopi...,strange squirrel activity neighbor hope help f...
7,7,183611885,Crystal Weaver,City Park,15 Apr,Fires around town. Someone just tried to set t...,fire town try set tree house vicinity fire gas...
9,9,184321665,Bryn Creager,Congress Park,21 Apr,Brake in. On 4/19/21 at 2:05 AM a man broke in...,brake man break car park 6th steal garage door...
10,10,184972043,Deb Christian,Hudson Street,27 Apr,It’s this whole thing of changing the housing ...,thing change housing code ask need
...,...,...,...,...,...,...,...
1079,624,171163993,Dyron Manning,Montbello,17 Dec,Lost bulldogs. My Bulldogs got out of back yar...,lose bulldog bulldog get yard area white brown...
1080,625,150824551,Lexy Beltran,Northeast Park Hill,6 Jun,Patio chairs. Any one selling patio chairs or ...,patio chair sell patio chair know
1081,626,141292402,News,NaN,NaN,Denver sets sights on hundreds of respite room...,set sight hundred respite room people experien...
1082,627,170010698,Leoma Burnette,Montbello,6 Dec,Lost dogs. Ringo & Trouble been missing since...,lose dog miss yesterday evening


___
___

# EDA ___
___
___

In [14]:
def count_containing(kwds,col,df):
    # will be a dataframe of post containing the search terms
    return_df = pd.DataFrame()
    # count of mentions
    m_count = 0 
    for word in kwds:
        posts = df[df[f'{col}'].str.lower().str.contains(word)].copy()
        return_df = return_df.append(posts)
        m_count += df[df[f'{col}'].str.lower().str.contains(f'{word}')].shape[0]
    
    return_df.drop_duplicates(inplace = True)
    kwd_mentions =(m_count/df.shape[0])*100
    print(f'''{m_count} posts containing{kwds}:
    {kwd_mentions} of Posts''')
    return return_df

In [15]:
homeless_words = [' homeless ',' unhoused ',' vagrant ',' squatter ',' sos ',' encampment ']
homeless_posts = count_containing(homeless_words, 'lemma', posts_df)

204 posts containing[' homeless ', ' unhoused ', ' vagrant ', ' squatter ', ' sos ', ' encampment ']:
    21.864951768488748 of Posts


We see above that we have the number of posts in our corpus containing words pertaining to homelessness!

With this function we will be able to inspect posts containing lists of keywords and return a count, percentage, and dataframe of posts containing the given words.

___
___
# Walmart Noise
___
___

In [16]:
walmart_words = ['walmart','wal-mart']

Nextdoor.com and Wal-mart have a bussiness partnership called neighbor helping neihgbor's intended to promote errand sharing durring the covid pandemic to assit the elderly or infirm. It would appear that as a product of this partnership, posts related to Walmart will appear no matter what term you query the Nextdoor site for. We need to look at how many of our posts that pertain to Walmart include mentions of Homelessness before we can decide how to handle these posts.

We are going to need to check for different spellings of the brand name , and to use the 'post' column, as Spacy does not recognize the name walmart and so will not have created lemma tokens for it.

## Count Walmart Mentions

In [17]:
walmart_posts = count_containing(walmart_words, 'post', posts_df)

48 posts containing['walmart', 'wal-mart']:
    5.144694533762058 of Posts


In [18]:
walmart_posts['post']

85      Keys found at Walmart... At Walmart Neighborho...
132     COVID-19 Vaccination Documentation. I was wond...
135     COVID vaccine. Just got an appointment for the...
228     Cat Food Delivered to wrong address. Did anyon...
234     free $20 Thanksgiving dinners from iBotta and ...
277     Jan needing assistance. I am starting this thr...
324     NON PROFIT TUESDAY. Hey All👋🏽👋🏽, im back at it...
329     Stolen Wallet from Car. So, I woke up to a cal...
334     Giving to Non-profits Tuesday. Hey All👋🏽👋🏽, im...
336     Non-profit. Hey All👋🏽👋🏽, im back at it again f...
350     Sodastream emergency! Where to exchange CO2 ca...
382     LIGHT FILLED💛MULTI LEVEL HOME for rent in Arva...
430     Items Found. Clorox bleach, Charmin toilet pap...
445     ATTENTION TO THOSE WHO SUFFER FROM SLEEP APNEA...
553     Walmart has appointments for the vax. Hey, nig...
569     Hi neighbors. I’m isaac. 👋. work for walmart D...
588     Labradoodles. Two of them just walked into Wal...
598     Questi

## Inspect Walmart posts for mentions of Homelessness

If mentions of the unhoused are fequently alongside mentions of Walmart than dropping all post including Walmart would alter our results in an undesirable way, Let's Check how many of the posts containing the 'Walmart' also pertain to the unhoused.

In [19]:
walmart_homeless_posts = count_containing(homeless_words, 'post', walmart_posts)

0 posts containing[' homeless ', ' unhoused ', ' vagrant ', ' squatter ', ' sos ', ' encampment ']:
    0.0 of Posts


In [20]:
walmart_posts.head(2)

,Unnamed: 0,post_id,author,location,date,post,lemma
85,85,181361739,T.L. Ma,Hale,25 Mar,Keys found at Walmart... At Walmart Neighborho...,key find keychain give cashier
132,132,179416233,Sherry Johnson,Crestmoor/Mayfair Park,8 Mar,COVID-19 Vaccination Documentation. I was wond...,wonder need letter doctor state co- morbidity ...


With such a small amount of overlap we can move forward confident that dropping all posts pertaining to Walmart is not likely to adversely affect our models reflection of reality in any way. We will now remove these posts from the DataFrame.

In [21]:
posts_df.shape[0]

933

In [22]:
posts_to_drop = pd.DataFrame()

In [23]:
posts_to_drop = posts_to_drop.append(walmart_posts)

In [24]:
def drop_posts(drop_words, col,df ):
    for word in drop_words:
        df = df[~df['post'].str.lower().str.contains(word)].copy()



def drop_posts(drop_ids, col,df ):
    for pid in drop_ids:
        for i, p in enumerate(df['post_id']):
            if pid == p:
                try:
                    df.drop(index = i, inplace = True)
                except:
                    pass
    return df

In [25]:
drop_ids = [ pid for pid in walmart_posts ]

In [26]:
posts_df.reset_index(inplace = True)
posts_df.shape

(933, 8)

___
___
# Pet Posts
___
___

Here we need to inspect the 'lemma' column as their are many variations of the words we are looking for and Spacy should have recognized them all.

In [27]:
posts_df.shape

(933, 8)

In [28]:
pet_words = [' cat ',' dog ',' pet '," cat's ", " dog's ", ' husky ',' kitty ',' kitten ']

The function we wrote for counting and removing walmart posts doesnt seem to work here. We will need to leave the topic of pets to our unsupervised steps.

In [29]:
pet_posts = count_containing(pet_words, 'lemma', posts_df)

177 posts containing[' cat ', ' dog ', ' pet ', " cat's ", " dog's ", ' husky ', ' kitty ', ' kitten ']:
    18.971061093247588 of Posts


In [30]:
pet_homeless_posts = count_containing(homeless_words, 'post', pet_posts)

22 posts containing[' homeless ', ' unhoused ', ' vagrant ', ' squatter ', ' sos ', ' encampment ']:
    15.492957746478872 of Posts


In [31]:
posts_to_drop = posts_to_drop.append(pet_posts)

In [32]:
posts_to_drop.shape[0]

190

In [33]:
posts_df.shape

(933, 8)

In [34]:
drop_post_id = [pid for pid in posts_to_drop['post_id']]

In [35]:
for i, post_id in enumerate(posts_df['post_id']):
    if post_id in drop_post_id:
        posts_df.drop(index = i, inplace = True)

In [36]:
posts_df

,index,Unnamed: 0,post_id,author,location,date,post,lemma
0,4,4,182941686,Beth Daoud,Congress Park,9 Apr,Squatters Camp. Am I the only one outraged by ...,outrage new squatter camp go 14th let allow ha...
2,7,7,183611885,Crystal Weaver,City Park,15 Apr,Fires around town. Someone just tried to set t...,fire town try set tree house vicinity fire gas...
3,9,9,184321665,Bryn Creager,Congress Park,21 Apr,Brake in. On 4/19/21 at 2:05 AM a man broke in...,brake man break car park 6th steal garage door...
4,10,10,184972043,Deb Christian,Hudson Street,27 Apr,It’s this whole thing of changing the housing ...,thing change housing code ask need
5,11,11,185372613,Doug Bohm,Congress Park,1 May,Homeless tent ablaze burning car & property. L...,homeless tent ablaze burn car property night a...
...,...,...,...,...,...,...,...,...
926,1077,622,133490918,Barbara Acker,Lowry,2 Jan 20,donations for refugees? Where can I donate odd...,donation refugee donate odd household item ref...
928,1079,624,171163993,Dyron Manning,Montbello,17 Dec,Lost bulldogs. My Bulldogs got out of back yar...,lose bulldog bulldog get yard area white brown...
929,1080,625,150824551,Lexy Beltran,Northeast Park Hill,6 Jun,Patio chairs. Any one selling patio chairs or ...,patio chair sell patio chair know
930,1081,626,141292402,News,NaN,NaN,Denver sets sights on hundreds of respite room...,set sight hundred respite room people experien...


In [37]:
posts_df.reset_index(inplace = True)

Initially the 13% of pet posts also containing homeless keywords made me hesitant to drop the posts outright, but later topic modeling showed that the pet content added too much noise for our model to adequately separate from the target content. We remove these posts from the corpus above.

___
___
# User Comments
___
___

Now we nee to merge the two comments dataframes and make sure that we remove all of the comments from posts about walmart that we removed before. Once this is finished we can export both of our cleaned dataframes. We will revisit this aspect of the process once we have assesed the results of our initial topic modeling! 

In [38]:
print(f'Shape Before Duplicate Dropping: {user_1_com.shape[0] + user_2_com.shape[0]}')

Shape Before Duplicate Dropping: 14698


In [39]:
user_comments = [user_1_com, user_2_com]
comments_df = pd.concat(user_comments, ignore_index=True)
comments_df.drop_duplicates(subset = 'comment', keep='last', inplace = True)

In [40]:
print(f'Shape Before Duplicate Dropping: {comments_df.shape[0]}')

Shape Before Duplicate Dropping: 7262


In [41]:
user_2_com

,Unnamed: 0,post_id,author,location,date,comment
0,0,186113992,Gabe Dusenbury,• Hale,3 days ago,"Not a great look, Park Hill!"
1,1,186113992,Andrew Walvoord,• Hilltop,3 days ago,Why not? Why should the city be putting homele...
2,2,186113992,Jessica Wood,• Northeast Park Hill,3 days ago,I think we should have least been asked before...
3,3,186113992,Michelle W.,• Sheridan,3 days ago,What would your answer have been?
4,4,186113992,Celine Garrett,• South Park Hill,3 days ago,It’s called compassion people.
...,...,...,...,...,...,...
7344,10,122730087,Cat S.,• Cheesman Park,1y,"If your building has a dumpster, you don't lik..."
7345,11,122730087,Trevor Price,• City Park,5 Sep 19,"If you are able to, you can also take it to AF..."
7346,12,122730087,Kama Newell,• City Park,5 Sep 19,You can schedule a pick up with Spring Back Co...
7347,13,122730087,Katrina Weidknecht,• Congress Park,5 Sep 19,Carol Altvater https://www.springbackco.org


We need to remove the comments from the walmart posts that we dropped before. We can use the posts_to_drop list that we made earlier.

In [42]:
# use the list of posts to drop from before.
comments_df = comments_df[~comments_df.post_id.isin(posts_to_drop)]
comments_df.reset_index(inplace = True)

In [43]:
comments_df.shape[0]

7262

Now we will create two columns in our posts data frame. One for the comments and another for post and comments together titled all_text. We will want to use all of the textual data from both the comments and the post for our topic model to account for the comments dialogues capacity to shift topics away from the content or the original post. 

In [45]:
posts_df['comments'] = np.nan

In [46]:
for i , post_id in enumerate(posts_df['post_id']):
    comments = ''
    for x, pid in enumerate(comments_df['post_id']):
        if post_id == pid:
            comments += comments_df['comment'][x] + '\n'
    posts_df['comments'][i] = comments

<ipython-input-46-855bea9906f1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts_df['comments'][i] = comments
/home/t0ad/anaconda3/envs/work/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [47]:
posts_df

,level_0,index,Unnamed: 0,post_id,author,location,date,post,lemma,comments
0,0,4,4,182941686,Beth Daoud,Congress Park,9 Apr,Squatters Camp. Am I the only one outraged by ...,outrage new squatter camp go 14th let allow ha...,"Everybody ""Wants to help"". But heaven forbid i..."
1,2,7,7,183611885,Crystal Weaver,City Park,15 Apr,Fires around town. Someone just tried to set t...,fire town try set tree house vicinity fire gas...,Thanks for the warning..that’s disturbing\nI’m...
2,3,9,9,184321665,Bryn Creager,Congress Park,21 Apr,Brake in. On 4/19/21 at 2:05 AM a man broke in...,brake man break car park 6th steal garage door...,Looks like homeless goy on 14th and columbine\...
3,4,10,10,184972043,Deb Christian,Hudson Street,27 Apr,It’s this whole thing of changing the housing ...,thing change housing code ask need,
4,5,11,11,185372613,Doug Bohm,Congress Park,1 May,Homeless tent ablaze burning car & property. L...,homeless tent ablaze burn car property night a...,Is this the same guy who panhandles on the sou...
...,...,...,...,...,...,...,...,...,...,...
742,926,1077,622,133490918,Barbara Acker,Lowry,2 Jan 20,donations for refugees? Where can I donate odd...,donation refugee donate odd household item ref...,
743,928,1079,624,171163993,Dyron Manning,Montbello,17 Dec,Lost bulldogs. My Bulldogs got out of back yar...,lose bulldog bulldog get yard area white brown...,
744,929,1080,625,150824551,Lexy Beltran,Northeast Park Hill,6 Jun,Patio chairs. Any one selling patio chairs or ...,patio chair sell patio chair know,
745,930,1081,626,141292402,News,NaN,NaN,Denver sets sights on hundreds of respite room...,set sight hundred respite room people experien...,"""We recognize there are also people who are pr..."


In [48]:
posts_df['all_text'] = posts_df['post'] + posts_df['comments']
comments = posts_df['comments']
comments = lemmatization(comments)
posts_df['all_lemma'] = [ com for com in comments ]

In [49]:
posts_df

,level_0,index,Unnamed: 0,post_id,author,location,date,post,lemma,comments,all_text,all_lemma
0,0,4,4,182941686,Beth Daoud,Congress Park,9 Apr,Squatters Camp. Am I the only one outraged by ...,outrage new squatter camp go 14th let allow ha...,"Everybody ""Wants to help"". But heaven forbid i...",Squatters Camp. Am I the only one outraged by ...,want help forbid show backyard want church exa...
1,2,7,7,183611885,Crystal Weaver,City Park,15 Apr,Fires around town. Someone just tried to set t...,fire town try set tree house vicinity fire gas...,Thanks for the warning..that’s disturbing\nI’m...,Fires around town. Someone just tried to set t...,thank warning disturbing glad neighbor injure ...
2,3,9,9,184321665,Bryn Creager,Congress Park,21 Apr,Brake in. On 4/19/21 at 2:05 AM a man broke in...,brake man break car park 6th steal garage door...,Looks like homeless goy on 14th and columbine\...,Brake in. On 4/19/21 at 2:05 AM a man broke in...,look homeless goy 14th sorry suck hope
3,4,10,10,184972043,Deb Christian,Hudson Street,27 Apr,It’s this whole thing of changing the housing ...,thing change housing code ask need,,It’s this whole thing of changing the housing ...,
4,5,11,11,185372613,Doug Bohm,Congress Park,1 May,Homeless tent ablaze burning car & property. L...,homeless tent ablaze burn car property night a...,Is this the same guy who panhandles on the sou...,Homeless tent ablaze burning car & property. L...,guy panhandle southwest intersection descripti...
...,...,...,...,...,...,...,...,...,...,...,...,...
742,926,1077,622,133490918,Barbara Acker,Lowry,2 Jan 20,donations for refugees? Where can I donate odd...,donation refugee donate odd household item ref...,,donations for refugees? Where can I donate odd...,
743,928,1079,624,171163993,Dyron Manning,Montbello,17 Dec,Lost bulldogs. My Bulldogs got out of back yar...,lose bulldog bulldog get yard area white brown...,,Lost bulldogs. My Bulldogs got out of back yar...,
744,929,1080,625,150824551,Lexy Beltran,Northeast Park Hill,6 Jun,Patio chairs. Any one selling patio chairs or ...,patio chair sell patio chair know,,Patio chairs. Any one selling patio chairs or ...,
745,930,1081,626,141292402,News,NaN,NaN,Denver sets sights on hundreds of respite room...,set sight hundred respite room people experien...,"""We recognize there are also people who are pr...",Denver sets sights on hundreds of respite room...,recognize people precariously house example ho...


___
___
# Export
___
___

In [50]:
posts_df.to_csv('../data/all_posts.csv', index = False)

In [51]:
comments_df.to_csv('../data/all_comments.csv', index = False)

___
___
___